## Two novel maze assembly comaprison

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(2, 2))
sessions = subjects.Tn().ratSday5
for sub, sess in enumerate(sessions):
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2

    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(maze1)
    assembly_maze2 = sess.replay.assemblyICA.getAssemblies(maze2)

    assemblies = np.hstack((assembly_maze1, assembly_maze2))
    assembly_corr = np.corrcoef(assemblies.T)
    np.fill_diagonal(assembly_corr, 0)

    ax = plt.subplot(gs[0])
    im = ax.imshow(assembly_corr, cmap="jet")
    ax.set_xlabel("Assembly index")
    ax.set_ylabel("Assembly index")
    plt.colorbar(im)
# endregion


## Cells selectivity for each environment (two novel maze)


In [ ]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig
from scipy import stats

In [14]:
figure = Fig()
fig, gs = figure.draw(num=1, grid=(3, 2))
sessions = subjects.Tn().ratSday5
for sub, sess in enumerate(sessions):
    pre=sess.epochs.pre
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2
    post2 = sess.epochs.post2

    spks = sess.spikes.pyr
    cell_ids = sess.spikes.pyrid
    nspikes_maze1 = np.asarray([np.histogram(cell, bins=maze1)[0] for cell in spks])
    nspikes_maze2 = np.asarray([np.histogram(cell, bins=maze2)[0] for cell in spks])

    #---- remove low firing cells --------
    good_cells_maze1 = cell_ids[np.where(nspikes_maze1>10)[0]]
    good_cells_maze2 = cell_ids[np.where(nspikes_maze2>10)[0]]
    good_cells_ids = np.intersect1d(good_cells_maze1,good_cells_maze2)

    #----- firing rate -------
    maze_dur = np.diff(maze1)+np.diff(maze2)
    good_cells_spks = sess.spikes.get_cells(good_cells_ids)
    frate = [np.histogram(cell,bins=maze1+maze2)[0][::2].sum()/maze_dur for cell in good_cells_spks]
    frate = np.asarray(frate)
    frate_color = frate-np.min(frate)/(np.max(frate)-np.min(frate))
    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(good_cells_ids,maze1)
    activation_maze1 = sess.replay.assemblyICA.getActivation(maze1)[0]
    assembly_maze2 = sess.replay.assemblyICA.getAssemblies(good_cells_ids,maze2)
    activation_maze2 = sess.replay.assemblyICA.getActivation(maze2)[0]

    mean_weight_maze1 = stats.zscore(np.mean(assembly_maze1,axis=1))
    mean_weight_maze2 = stats.zscore(np.mean(assembly_maze2,axis=1))

    # assemblies = np.hstack((assembly_maze1, assembly_maze2))
    # assembly_corr = np.corrcoef(assemblies.T)
    # np.fill_diagonal(assembly_corr, 0)

    ax = plt.subplot(gs[0])
    ax.spines.left.set_position('zero')
    ax.spines.bottom.set_position('zero')
    ax.scatter(mean_weight_maze1,mean_weight_maze2,s=frate*30,c=frate_color,cmap='coolwarm')
    # ax.set_xlabel("maze1")
    # ax.set_ylabel("maze2")
    # ax.set_aspect('equal', adjustable='datalim')
    # ax.set_xlim([-0.1,0.1])
    # ax.set_xlim([-0.1,0.1])
    # ax.set_xticks([-0.05,0,0.05])
    # ax.set_yticks([-0.05,0,0.05])
    # plt.colorbar(im)

    ax = plt.subplot(gs[1])
    frate_maze1 = sess.spikes.firing_rate(good_cells_ids,maze1)
    frate_maze2 = sess.spikes.firing_rate(good_cells_ids,maze2)
    ax.scatter(frate_maze1,frate_maze2,c=frate_color,cmap='coolwarm')
    ax.set_xscale('log')
    ax.set_yscale('log')

    ax = plt.subplot(gs[2])
    corr,time = sess.spikes.corr.across_time_window(good_cells_ids,period=[pre[0],post2[1]])
    np.fill_diagonal(corr,val=0)
    time = time/3600
    im = ax.pcolormesh(time,time,corr,shading = 'gouraud',cmap='jet')
    ax2 = plt.subplot(gs[3])
    plt.colorbar(im,ax=ax2)



## Activation strength of maze1 plotted on maze2 and show ripple periods
* Also compare mean activation in non-ripple periods vs ripple-periods. Then compare that with activation of maze2 assemblies. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns
import subjects
from plotUtil import Fig

In [18]:
%matplotlib qt
figure = Fig()
fig,gs = figure.draw(num=1,grid=(3,4))
sessions = subjects.Tn().ratSday5
for sub,sess in enumerate(sessions):
    rpls = sess.ripple.events
    pyrid = sess.spikes.pyrid
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2

    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(pyrid,maze1)
    activation_maze2,t_maze2 = sess.replay.assemblyICA.getActivation(period=maze2)

    mean_activation = np.mean(activation_maze2,axis=0)

    #--- plotting mean reactivation with ripple epochs------

    ax = plt.subplot(gs[0,:])
    ax.plot(t_maze2[:-1],mean_activation,'k')
    sess.ripple.plot_ripples(maze2,ax)
    ax.set_xlabel('Maze2 time (seconds)')
    ax.set_ylabel('Mean activation strength')
    ax.legend(['','ripples'])

    # ------- mean reactivation within ripple and outside ripple --------
    rpls_maze2 = rpls[(rpls.start>maze2[0])&(rpls.end<maze2[1])]
    rpls_maze2 = rpls_maze2[['start','end']].to_numpy().flatten()
    mean_act_rpl = stats.binned_statistic(t_maze2[1:],mean_activation,bins=rpls_maze2)[0]
    mean_act_rpl = np.nan_to_num(mean_act_rpl,nan=0)

    mean_within_rpl = mean_act_rpl[::2]
    mean_outside_rpl = mean_act_rpl[1::2]
    data1 = pd.DataFrame({'reactivation':np.hstack((mean_within_rpl,mean_outside_rpl)),
    'loc':['rpl']*len(mean_within_rpl)+['out_rpl']*len(mean_outside_rpl),
    'maze':'maze1'})

    # ax= plt.subplot(gs[1,0])
    # # ax.bar(['rpl','out_rpl'],[mean_within_rpl,mean_outside_rpl])
    # sns.barplot(data=data,x='loc',y='reactivation',ax=ax)
    # ax.set_xlim([0,1])

    #---- maze2 on maze2 ------------
    assembly_maze2 = sess.replay.assemblyICA.getAssemblies(pyrid,maze2)
    activation_maze2,t_maze2 = sess.replay.assemblyICA.getActivation(period=maze2)

    mean_activation = np.mean(activation_maze2,axis=0)

    # ------- mean reactivation within ripple and outside ripple --------
    rpls_maze2 = rpls[(rpls.start>maze2[0])&(rpls.end<maze2[1])]
    rpls_maze2 = rpls_maze2[['start','end']].to_numpy().flatten()
    mean_act_rpl = stats.binned_statistic(t_maze2[1:],mean_activation,bins=rpls_maze2)[0]
    mean_act_rpl = np.nan_to_num(mean_act_rpl,nan=0)

    mean_within_rpl = mean_act_rpl[::2]
    mean_outside_rpl = mean_act_rpl[1::2]
    data2 = pd.DataFrame({'reactivation':np.hstack((mean_within_rpl,mean_outside_rpl)),
    'loc':['rpl']*len(mean_within_rpl)+['out_rpl']*len(mean_outside_rpl),
    'maze':'maze2'})

    data = data1.append(data2)

    
    ax= plt.subplot(gs[1,0])
    # ax.bar(['rpl','out_rpl'],[mean_within_rpl,mean_outside_rpl])
    sns.barplot(data=data,x='loc',y='reactivation',ax=ax,palette='Pastel2',ec='k',hue='maze')
    ax.tick_params(axis='x',labelrotation=45)
    # ax.set_ylim([0,1])

    

    

In [19]:

figure.savefig('ica_maze1_maze2')
